In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import networkx as nx
from networkx.algorithms import bipartite
# import community
from networkx.readwrite import json_graph
# import nx_altair as nxa
from networkx.algorithms.community import greedy_modularity_communities
from pyvis import network as net
# from node2vec import Node2Vec
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import itertools
import collections
from tqdm.notebook import trange, tqdm

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("..")
from network_analysis.load_datasets import get_updated_shxco_data
from network_analysis.generate_network_metrics import *
from network_analysis.create_networks import *
from network_analysis.read_write_networks import * 

Exploring the stability of graphs in these groupings:
- compare overall graph density
- compare node metrics
1. All Events **vs** All Events minus Exceptional Ones **vs** All Events minus those of Killen and Raphael France
   1. Bipartite
      1. All time
      2. Seasonal
      3. 1920s vs 1930s
   2. Unipartite
      1. All time
      2. Seasonal
      3. 1920s vs 1930s

Goals:
- identify communities
- identify shape of network and how we want to divide it for analysis
- find out which metrics are most useful for modeling subscribers and book reading habits

#### Baseline datasets

In [3]:
members_df, books_df, borrow_events, events_df = get_updated_shxco_data()

In [4]:
all_events = borrow_events.copy()

unexceptional_events = borrow_events[borrow_events.exceptional_types.isna()]

no_rk_borrow_events = borrow_events[(borrow_events.member_id != 'killen') & (borrow_events.member_id != 'raphael-france')]
no_rk_members_df = members_df[(members_df.member_id != 'killen') & (members_df.member_id != 'raphael-france')]


### Bipartite Comparisons

#### Comparing Across Entire Time of Sco Library

In [5]:
member_attrs = {'uri': 'member_id'}
book_attrs = {'uri': 'item_uri'}
edge_attrs = {'weight': 'counts'}
all_events_grouped = all_events.groupby(['member_id', 'item_uri']).size().reset_index(name='counts')
unexceptional_events_grouped = unexceptional_events.groupby(['member_id', 'item_uri']).size().reset_index(name='counts')
no_rk_events_grouped = no_rk_borrow_events.groupby(['member_id', 'item_uri']).size().reset_index(name='counts')
should_process = True
write_to_file = True
sk_metrics = ['katz', 'louvain']
link_metrics = ['HITS', 'CoHITS', 'BiRank', 'BGRM']


all_events_bipartite_graph, all_events_bipartite_nodelist, all_events_bipartite_edgelist, all_events_members, all_events_books = check_reload_build_bipartite_graphs(all_events_grouped, member_attrs, book_attrs, edge_attrs, should_process, write_to_file, './data/all_events_bipartite', sk_metrics, link_metrics, members_df, books_df)

unexceptional_bipartite_graph, unexceptional_bipartite_nodelist, unexceptional_bipartite_edgelist, unexceptional_members, unexceptional_books = check_reload_build_bipartite_graphs(unexceptional_events_grouped, member_attrs, book_attrs, edge_attrs, should_process, write_to_file,'./data/unexceptional_bipartite', sk_metrics, link_metrics, members_df, books_df)

no_rk_bipartite_graph, no_rk_bipartite_nodelist, no_rk_bipartite_edgelist, no_rk_members, no_rk_books = check_reload_build_bipartite_graphs(no_rk_events_grouped, member_attrs, book_attrs, edge_attrs, should_process, write_to_file, './data/no_rk_bipartite', sk_metrics, link_metrics, members_df, books_df)


reloading saved graph: ./data/all_events_bipartite
reloading saved graph: ./data/unexceptional_bipartite
reloading saved graph: ./data/no_rk_bipartite


#### Correlations Between Bipartite Graph Metrics

In [6]:
all_events_members = all_events_members.drop('borrow_count', axis=1)
unexceptional_members = unexceptional_members.drop('borrow_count', axis=1)
no_rk_members = no_rk_members.drop('borrow_count', axis=1)


In [7]:
all_borrows = all_events.groupby(
    'member_id').size().reset_index(name='borrow_count')

unexceptional_borrows = unexceptional_events.groupby(
    'member_id').size().reset_index(name='borrow_count')
no_rk_borrows = no_rk_borrow_events.groupby(
    'member_id').size().reset_index(name='borrow_count')

all_events_members = pd.merge(all_borrows, all_events_members, on='member_id')
unexceptional_members = pd.merge(
    unexceptional_borrows, unexceptional_members, on='member_id')
no_rk_members = pd.merge(no_rk_borrows, no_rk_members, on='member_id')



In [8]:
columns = no_rk_books.columns.to_list()
columns = [c for c in columns if ('local' in c) | ('global' in c)]
no_rk_books_corr = no_rk_books[columns].corr()
all_events_books_corr = all_events_books[columns].corr()
unexceptional_books_corr = unexceptional_books[columns].corr()
no_rk_members_corr = no_rk_members[columns].corr()
all_events_members_corr = all_events_members[columns].corr()
unexceptional_members_corr = unexceptional_members[columns].corr()


def generate_corr_chart(corr_df, title):
    # data preparation
    pivot_cols = list(corr_df.columns)
    corr_df['cat'] = corr_df.index

    base = alt.Chart(corr_df).transform_fold(pivot_cols).encode(
        x="cat:N",  y='key:N').properties(height=300, width=300, title=title)
    boxes = base.mark_rect().encode(color=alt.Color(
        "value:Q", scale=alt.Scale(scheme="redyellowblue")))
    labels = base.mark_text(size=5, color="grey").encode(
        text=alt.Text("value:Q", format="0.1f"))
    chart = boxes + labels
    return chart

In [9]:
unexceptional_corr_members_chart = generate_corr_chart(
    unexceptional_members_corr, 'member correlations for unexceptional data')
no_rk_corr_members_chart = generate_corr_chart(
    no_rk_members_corr, 'member correlations for no rk data')
all_events_corr_members_chart = generate_corr_chart(
    all_events_members_corr, 'member correlations for all events data')

alt.hconcat(*[all_events_corr_members_chart,
            unexceptional_corr_members_chart, no_rk_corr_members_chart])


alt.HConcatChart(...)

In [10]:
unexceptional_corr_books_chart = generate_corr_chart(
    unexceptional_books_corr, 'book correlations for unexceptional data')
no_rk_corr_books_chart = generate_corr_chart(
    no_rk_books_corr, 'book correlations for no rk data')
all_events_corr_books_chart = generate_corr_chart(
    all_events_books_corr, 'book correlations for all events data')

alt.hconcat(*[all_events_corr_books_chart,
            unexceptional_corr_books_chart, no_rk_corr_books_chart])


alt.HConcatChart(...)

In [11]:
melted_all_events_members = pd.melt(all_events_members_corr, id_vars=['cat'],value_vars=columns)
melted_unex_events_members = pd.melt(unexceptional_members_corr, id_vars=['cat'], value_vars=columns)
melted_all_events_members['updated_variable'] = melted_all_events_members['cat'] + ' / ' + melted_all_events_members['variable']
melted_all_events_members['type'] = 'all_events_members'
melted_unex_events_members['updated_variable'] = melted_unex_events_members['cat'] + \
    ' / ' + melted_unex_events_members['variable']
melted_unex_events_members['type'] = 'unexceptional_events_members'


In [12]:
melted_all_events_books = pd.melt(all_events_books_corr, id_vars=[
                                    'cat'], value_vars=columns)
melted_unex_events_books = pd.melt(unexceptional_books_corr, id_vars=[
                                     'cat'], value_vars=columns)
melted_all_events_books['updated_variable'] = melted_all_events_books['cat'] + \
    ' / ' + melted_all_events_books['variable']
melted_all_events_books['type'] = 'all_events_books'
melted_unex_events_books['updated_variable'] = melted_unex_events_books['cat'] + \
    ' / ' + melted_unex_events_books['variable']
melted_unex_events_books['type'] = 'unexceptional_events_books'


In [13]:
concat_corr_members = pd.concat([melted_all_events_members, melted_unex_events_members])

pivot_corr_members = pd.pivot(concat_corr_members, index=['updated_variable', 'cat', 'variable'], columns='type', values='value').reset_index()

concat_corr_books = pd.concat([melted_all_events_books, melted_unex_events_books])

pivot_corr_books = pd.pivot(concat_corr_books, index=['updated_variable', 'cat', 'variable'], columns='type', values='value').reset_index()
selection = alt.selection_multi(fields=['cat'], bind='legend')
chart_members = alt.Chart(pivot_corr_members).mark_circle().encode(
    x='unexceptional_events_members:Q',
    y='all_events_members:Q',
    color='cat:N',
    tooltip=['updated_variable', 'cat', 'variable', 'all_events_members', 'unexceptional_events_members'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)
chart_books = alt.Chart(pivot_corr_books).mark_circle().encode(
    x='unexceptional_events_books:Q',
    y='all_events_books:Q',
    color=alt.Color('cat:N', scale=alt.Scale(scheme="category20b")),
    tooltip=['updated_variable', 'cat', 'variable',
             'all_events_books', 'unexceptional_events_books'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)
alt.vconcat(chart_members, chart_books)


alt.VConcatChart(...)

In [14]:
len(all_events_members), len(unexceptional_members), len(no_rk_members)

(536, 508, 534)

In [15]:
partial_df = pd.read_csv('../dataset_generator/data/partial_borrowers.csv')
partial_df[0:1]


,member_id,subscription_start,subscription_end,known_borrows
0,martin-maud,1927-03-19,1927-06-19,36


In [29]:
members_all = all_events_members.uri.unique().tolist()
members_unex = unexceptional_members.uri.unique().tolist()
partial_members = partial_df.member_id.unique().tolist()
exceptional_only = list(set(members_all) - set(members_unex))
print('exceptional only', exceptional_only, 'partial and exceptional',set(partial_members) & set(exceptional_only))


exceptional only ['wright-ellen', 'goyert', 'lebois', 'faidherbe', 'melot', 'schueller', 'mayran', 'saillet', 'hartmann', 'pauleau', 'tourneux', 'samyn', 'wigram', 'leveque-susanne', 'henry-georgette', 'clermont-tonnerre', 'chanler', 'paige', 'auger', 'wallace-lillian', 'saby', 'harvey-dorothy-dudley', 'watson-capt', 'marcel-gabriel', 'tehin', 'zibell', 'du-bos', 'michaux-henri'] partial and exceptional set()


In [30]:
members = all_events_members.copy()
members['type'] = 'all_events'
members_unex = unexceptional_members.copy()
members_unex['type'] = 'unexceptional'
members.loc[(members.exceptional_types.isna() == True), 'is_exceptional'] = False
members.loc[(members.exceptional_types.isna() == False), 'is_exceptional'] = True
members.loc[(members.member_id.isin(partial_members)), 'is_partial'] = True
members.loc[(members.member_id.isin(partial_members) == False), 'is_partial'] = False

members_unex.loc[(
    members_unex.exceptional_types.isna() == True), 'is_exceptional'] = False
members_unex.loc[(
    members_unex.exceptional_types.isna() == False), 'is_exceptional'] = True
members_unex.loc[(members_unex.member_id.isin(
    partial_members)), 'is_partial'] = True
members_unex.loc[(members_unex.member_id.isin(
    partial_members) == False), 'is_partial'] = False


In [31]:
memb_cols = members.columns
memb_cols = [c for c in memb_cols if ('local' in c) | ('global' in c)]
memb_cols.remove('local_louvain')
memb_cols.remove('global_louvain')
a, b = 0, 100

for col in memb_cols:
    members[f'norm_{col}'] = members[[f'{col}']].apply(
        lambda x: (x-x.min())/(x.max()-x.min()) * (b - a) + a)
    members_unex[f'norm_{col}'] = members_unex[[f'{col}']].apply(
        lambda x: (x-x.min())/(x.max()-x.min()) * (b - a) + a)


In [32]:
var_cols = ['global_degree','local_degree', 'global_clustering', 'local_clustering',
            'global_closeness', 'local_closeness', 'global_betweenness',
            'local_betweenness', 'local_katz', 'local_HITS', 'local_CoHITS', 'local_BiRank',
            'local_BGRM', 'global_katz', 'global_HITS',
            'global_CoHITS', 'global_BiRank', 'global_BGRM']
norm_cols = [f'norm_{c}' for c in var_cols]
joined_members = pd.concat([members, members_unex])
concat_members = joined_members[['is_exceptional', 'is_partial', 'type', 'uri'] + norm_cols]


In [33]:
melted_members = pd.melt(concat_members, id_vars=['uri', 'type', 'is_exceptional', 'is_partial'],value_vars=norm_cols)


In [34]:
pivot_members = pd.pivot(melted_members, index=['uri', 'variable', 'is_partial', 'is_exceptional'], columns='type', values='value').reset_index()


In [35]:
import statsmodels.api as sm
member_ids = pivot_members[pivot_members.unexceptional.isna() == False].uri.unique().tolist()
for member_id in member_ids:
    member_df = pivot_members[pivot_members.uri == member_id]
    
    X = member_df["unexceptional"]
    y = member_df['all_events']

    # Note the difference in argument order
    model = sm.OLS(y, X).fit()
    predictions = model.predict(X)  # make the predictions by the model

    # Print out the statistics
    if round(model.rsquared, 2) < 0.70:
        print(member_id,  f'{model.rsquared:.2f}')
        print(joined_members[joined_members.uri == member_id].component.values)


carr-philip 0.28
[0 3]
cayeux 0.25
[0 1]
chenneviere 0.07
[0 4]
church-barbara 0.49
[0 0]
fischer-marjorie 0.60
[0 0]
franchot 0.66
[0 0]
gibault 0.31
[0 8]
giedion-welcker 0.69
[0 0]
lamberts 0.57
[0 0]
leer 0.62
[0 0]
mingalon 0.68
[0 0]
murphy-dudley 0.16
[ 0 12]
porel 0.43
[0 0]
rogers-samuel 0.58
[0 0]
tate 0.42
[0 0]
tree 0.10
[0 0]
treirse 0.30
[ 0 13]
venable 0.56
[0 0]
watson-sarah-pressly 0.69
[0 0]
wilkinson-tudor 0.62
[0 0]
wilson-natalie 0.66
[0 0]


In [336]:
exceptional_only = pivot_members[(pivot_members.unexceptional.isna()) | (pivot_members.uri == 'killen')]
selection = alt.selection_multi(fields=['variable'], bind='legend')
alt.Chart(exceptional_only).mark_bar().encode(
    x='all_events:Q', 
    y=alt.Y('variable:N', axis=alt.Axis(title='variable', labels=False)), 
    color=alt.Color('variable:N', scale=alt.Scale(scheme='plasma')),
    facet=alt.Facet('uri:N', columns=4),
    tooltip=['uri', 'variable', 'is_partial', 'is_exceptional', 'all_events'], 
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title='Exceptional only',
    width=75,
    height=75
)


alt.Chart(...)

In [36]:
# alt.Chart(pivot_members).mark_circle().encode(
#     x='unexceptional:Q',
#     y='all_events:Q',
#     color='variable:N',
#     # tooltip=['uri', 'variable', 'is_partial', 'is_exceptional', 'value']
#     facet=alt.Facet('uri:N', columns=5)
# ).properties(
#     width=75,
#     height=75
# )



In [206]:
# alt.data_transformers.disable_max_rows()
# field_1 = 'unexceptional'
# field_2 = 'all_events'
# base = alt.Chart(pivot_members).encode(
#     x=f'{field_1}:Q',
#     y=f'{field_2}:Q',
# ).properties(
#     height=50,
#     width=50,
# )

# selection = alt.selection_single(fields=['variable'])
# chart = alt.layer(
#     base.mark_circle().encode(
#         opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
#         tooltip=['variable', f'{field_1}', f'{field_2}']
#     ),
#     base.transform_regression(f'{field_1}', f'{field_2}').mark_line()
# ).add_selection(selection).facet(
#     columns=5,
#     row='uri'
# ).properties(title='Comparing Graph Metrics for Each Member by Seasons and Year').resolve_scale(x='independent', y='independent')

# chart


### Unipartite Comparisons

#### Comparing Across Entire Time of Sco Library

In [37]:
member_attrs = {'uri': 'member_id'}
book_attrs = {'uri': 'item_uri'}
edge_attrs = {'weight': 'counts'}
node_attrs = {}
all_events_grouped = all_events.groupby(
    ['member_id', 'item_uri']).size().reset_index(name='counts')
unexceptional_events_grouped = unexceptional_events.groupby(
    ['member_id', 'item_uri']).size().reset_index(name='counts')
no_rk_events_grouped = no_rk_borrow_events.groupby(
    ['member_id', 'item_uri']).size().reset_index(name='counts')
should_process = True
write_to_file = True
sk_metrics = ['katz', 'louvain']
link_metrics = ['pagerank', 'hubs', 'auth']
is_projected = True


projected_members_graph, projected_members_nodelist, projected_members_edgelist, projected_books_graph, projected_books_nodelist, projected_books_edgelist, projected_members, projected_books = check_reload_build_unipartite_graphs(
    all_events_grouped, all_events, member_attrs, book_attrs, edge_attrs, node_attrs, should_process, write_to_file, './data/all_events_unipartite_projected', sk_metrics, link_metrics, members_df, books_df, is_projected)

unexceptional_projected_members_graph, unexceptional_projected_members_nodelist, unexceptional_projected_members_edgelist, unexceptional_projected_books_graph, unexceptional_projected_books_nodelist, unexceptional_projected_books_edgelist, unexceptional_projected_members, unexceptional_projected_books = check_reload_build_unipartite_graphs(
    unexceptional_events_grouped, unexceptional_events, member_attrs, book_attrs, edge_attrs, node_attrs, should_process, write_to_file, './data/unexceptional_events_unipartite_projected', sk_metrics, link_metrics, members_df, books_df, is_projected)


reloading saved graph: ./data/all_events_unipartite_projected
reloading saved graph: ./data/unexceptional_events_unipartite_projected


In [40]:
projected_members = projected_members.drop('borrow_count', axis=1)
unexceptional_projected_members = unexceptional_projected_members.drop('borrow_count', axis=1)


In [41]:
all_borrows = all_events.groupby(
    'member_id').size().reset_index(name='borrow_count')

unexceptional_borrows = unexceptional_events.groupby(
    'member_id').size().reset_index(name='borrow_count')

projected_members = pd.merge(all_borrows, projected_members, on='member_id')
unexceptional_projected_members = pd.merge(
    unexceptional_borrows, unexceptional_projected_members, on='member_id')



#### Correlations Between Unipartite Graph Metrics

In [44]:
columns = ['global_pagerank', 'global_hubs', 'global_auth',
           'local_pagerank', 'local_hubs', 'local_auth', 
           'global_degree', 'local_degree', 'global_eigenvector',
           'local_eigenvector', 'global_closeness', 'local_closeness',
           'global_betweenness', 'local_betweenness', 'global_clustering',
           'local_clustering', 'global_graph_radius',
           'global_diameter', 'local_graph_radius', 'local_diameter', 
           'local_katz', 'global_katz', 'borrow_count']
members_corr = projected_members[columns].corr()
books_corr = projected_books[columns].corr()
unexceptional_projected_members_corr = unexceptional_projected_members[columns].corr(
)
unexceptional_projected_books_corr = unexceptional_projected_books[columns].corr(
)

def generate_corr_chart(corr_df, title):
    # data preparation
    pivot_cols = list(corr_df.columns)
    corr_df['cat'] = corr_df.index

    base = alt.Chart(corr_df).transform_fold(pivot_cols).encode(
        x="cat:N",  y='key:N').properties(height=300, width=300, title=title)
    boxes = base.mark_rect().encode(color=alt.Color(
        "value:Q", scale=alt.Scale(scheme="redyellowblue")))
    labels = base.mark_text(size=5, color="grey").encode(
        text=alt.Text("value:Q", format="0.1f"))
    chart = boxes + labels
    return chart


In [45]:

members_chart = generate_corr_chart(members_corr, 'unipartite members all data correlations')
books_chart = generate_corr_chart(
    books_corr, 'unipartite books all data correlations')
unexceptional_projected_members_chart = generate_corr_chart(
    unexceptional_projected_members_corr, 'unipartite unexceptional_projected_members all data correlations')
unexceptional_projected_books_chart = generate_corr_chart(
    unexceptional_projected_books_corr, 'unipartite unexceptional_projected_books all data correlations')
alt.hconcat(*[members_chart, unexceptional_projected_members_chart,
            books_chart, unexceptional_projected_books_chart])


alt.HConcatChart(...)

In [46]:
melted_members = pd.melt(members_corr, id_vars=['cat'], value_vars=columns)
melted_unex_members = pd.melt(unexceptional_projected_members_corr, id_vars=['cat'], value_vars=columns)
melted_members['updated_variable'] = melted_members['cat'] + \
    ' / ' + melted_members['variable']
melted_members['type'] = 'all_events_members'
melted_unex_members['updated_variable'] = melted_unex_members['cat'] + \
    ' / ' + melted_unex_members['variable']
melted_unex_members['type'] = 'unexceptional_events_members'


In [47]:
melted_books = pd.melt(books_corr, id_vars=[
    'cat'], value_vars=columns)
melted_unex_books = pd.melt(unexceptional_projected_books_corr, id_vars=[
    'cat'], value_vars=columns)
melted_books['updated_variable'] = melted_books['cat'] + \
    ' / ' + melted_books['variable']
melted_books['type'] = 'all_events_books'
melted_unex_books['updated_variable'] = melted_unex_books['cat'] + \
    ' / ' + melted_unex_books['variable']
melted_unex_books['type'] = 'unexceptional_events_books'


In [48]:
concat_corr_members = pd.concat([melted_members, melted_unex_members])

pivot_corr_members = pd.pivot(concat_corr_members, index=['updated_variable', 'cat', 'variable'], columns='type', values='value').reset_index()

concat_corr_books = pd.concat([melted_books, melted_unex_books])

pivot_corr_books = pd.pivot(concat_corr_books, index=['updated_variable', 'cat', 'variable'], columns='type', values='value').reset_index()
selection = alt.selection_multi(fields=['cat'], bind='legend')
chart_members = alt.Chart(pivot_corr_members).mark_circle().encode(
    x='unexceptional_events_members:Q',
    y='all_events_members:Q',
    color='cat:N',
    tooltip=['updated_variable', 'cat', 'variable', 'all_events_members', 'unexceptional_events_members'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)
chart_books = alt.Chart(pivot_corr_books).mark_circle().encode(
    x='unexceptional_events_books:Q',
    y='all_events_books:Q',
    color=alt.Color('cat:N', scale=alt.Scale(scheme="category20b")),
    tooltip=['updated_variable', 'cat', 'variable',
             'all_events_books', 'unexceptional_events_books'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).add_selection(
    selection
)
alt.vconcat(chart_members, chart_books)


alt.VConcatChart(...)

In [ ]:
# is_projected = False
# unprojected_members_graph, unprojected_members_nodelist, unprojected_members_edgelist, unprojected_books_graph, unprojected_books_nodelist, unprojected_books_edgelist, unprojected_members, unprojected_books = check_reload_build_unipartite_graphs(
#     unexceptional_events_grouped, unexceptional_events, member_attrs, book_attrs, edge_attrs, node_attrs, should_process, write_to_file, './data/all_events_unipartite_unprojected', sk_metrics, link_metrics, members_df, books_df, is_projected)


### Temporal Network Comparisons

#### Bipartite Comparisons

In [ ]:
# Process the borrowers graph through the network metrics and return dataframes for each month and each year
years = all_events.year.unique().tolist()
books_dfs = []
members_dfs = []
month_ranges = [{'Winter': [12, 1, 2]}, {'Spring': [3, 4, 5]},
                {'Summer': [6, 7, 8]}, {'Fall': [9, 10, 11]}]
for year in tqdm(years):
    for month in month_ranges:
        # Make books graph for each month range
        member_attrs = {'uri': 'member_id'}
        book_attrs = {'uri': 'item_uri'}
        edge_attrs = {'weight': 'counts'}
        all_events_grouped = all_events[(all_events.year == year) & (all_events.month.isin(list(month.values())[0]))].groupby(['member_id', 'item_uri']).size().reset_index(name='counts')
        should_process = True
        write_to_file = False
        sk_metrics = ['katz', 'louvain']
        link_metrics = ['HITS', 'CoHITS', 'BiRank', 'BGRM']


        all_events_bipartite_graph, all_events_bipartite_nodelist, all_events_bipartite_edgelist, all_events_members, all_events_books = check_reload_build_bipartite_graphs(all_events_grouped, member_attrs, book_attrs, edge_attrs, should_process, write_to_file, './data/all_events_bipartite', sk_metrics, link_metrics, members_df, books_df)
        all_event_books['seasons'] = list(month.keys())[0]
        all_events_members['seasons'] = list(month.keys())[0]
        books_dfs.append(all_events_books)
        members_dfs.append(all_events_members)
        
